to look at interesting cases for rates of decay, we need to be able to have complex coefficients; however, I haven't been able to get sage to work using rationals instead of floats, so here we are


In [13]:
#setup; don't mess with this
C.<z1,z2>=PolynomialRing(CC,order='negdeglex') #defining the ring; wanted to do rationals adjoin i, but couldn't get it to work so we're stuck with decimals
def buildmonoms(howmany):
    C.<z1,z2>=PolynomialRing(CC,order='negdeglex')
    monomslist=[]
    m=0
    for totaldeg in range(0,howmany+1):
        k=0;l=totaldeg;
        while k<=totaldeg and m<=howmany:
            monomslist.append(sage_eval('z1^'+str(l)+'*z2^'+str(k),locals=vars()))#so it's not just strings
            #print (k,l),newcoefs[(k, l)]
            m=m+1
            k=k+1
            l=l-1
    return monomslist
def multideg(monom): #take a monomial in z1 and z2 and return an ordered pair with its multidegree 
                    #(there's probably a way to do this more generally)
    C.<z1,z2>=PolynomialRing(CC,order='negdeglex')
    z1deg=monom.degree(z1)
    z2deg=monom.degree(z2)
    return (z1deg,z2deg)
def coeffdict(poly,monomslist): #takes a polynomial (as an expression) and a pregenerated list of monomials and returns a dictionary
                                # keys are ordered pairs of degrees of a monomial, entries are coefficients
    polydict={}
    for monom in monomslist:
        multi=multideg(monom)
        #print(multi)
        d={z1:multi[0],z2:multi[1]}
        #print(d)
        polydict.update({multideg(monom):poly.coefficient(d)})
    return polydict
#inner product
def InnerProd(f,g,monomslist):
    cfsf=coeffdict(f,monomslist)#the coefficients paired into a dictionary with the multi-indices
    cfsg=coeffdict(g,monomslist)
    #print cfsf
    #print cfsg
    pairs=list(set(list(cfsf.keys())+list(cfsg.keys())))
    ip=0
    for alpha in pairs:
        term=cfsf[alpha]*conjugate(cfsg[alpha])
        ip=ip+term
    return ip
#
def buildMatrix(monomslist,fcn,whichpn):
    M=matrix(CC,whichpn+1,lambda x,y: InnerProd(monomslist[y]*fcn,monomslist[x]*fcn,monomslist))
    return M
def RHSmateq(f,whichpn):
    fcn(z1,z2)=f
    RHSveclist=[conjugate(f(0,0))]
    for n in range(whichpn):
        RHSveclist=RHSveclist+[0]
    return vector(RHSveclist)
# puts the coefficients from a list into the polynomial
def coeffpoly(optimal_list,whichOA,monomslist):
    thepoly='0'
    for coef in range(0,whichOA+1):
            thepoly=thepoly+'+'+str(optimal_list[coef])+'*'+str(monomslist[coef])
            #print (k,l),newcoefs[(k, l)]

    return sage_eval(thepoly,locals={'z1':z1,'z2':z2})

In [14]:
#getting a list of monomials
mlist=buildmonoms(50) #you need to have at least as many monomials as optimal approximants, make sure you understand why
print(mlist)

[1.00000000000000, z1, z2, z1^2, z1*z2, z2^2, z1^3, z1^2*z2, z1*z2^2, z2^3, z1^4, z1^3*z2, z1^2*z2^2, z1*z2^3, z2^4, z1^5, z1^4*z2, z1^3*z2^2, z1^2*z2^3, z1*z2^4, z2^5, z1^6, z1^5*z2, z1^4*z2^2, z1^3*z2^3, z1^2*z2^4, z1*z2^5, z2^6, z1^7, z1^6*z2, z1^5*z2^2, z1^4*z2^3, z1^3*z2^4, z1^2*z2^5, z1*z2^6, z2^7, z1^8, z1^7*z2, z1^6*z2^2, z1^5*z2^3, z1^4*z2^4, z1^3*z2^5, z1^2*z2^6, z1*z2^7, z2^8, z1^9, z1^8*z2, z1^7*z2^2, z1^6*z2^3, z1^5*z2^4, z1^4*z2^5]


In [15]:
#Function zoo #fill in the other functions we discussed
p1=1-(z1+z2)/2
p2=4 - 3*z1 - z2 - z1*z2 +z1^2
p3=2*z1*z2 + (-1+i)*(z1+z2) - 2*i
p4=(z1+1)*p1
p5= i - (z1+z2)/2
#show(p1)

In [16]:
#Computing the OPAs and optimal norms
f = p5
howmany=50 #how many optimal approximants you need to compute
opa=[]
OptNorms=[]
for whichOA in range(0,howmany):
    JAMmat=buildMatrix(mlist,f,whichOA)
    #print(JAMmat)
    optimal_coeffs=list(JAMmat.solve_right(RHSmateq(f,whichOA)))
    #print(optimal_coeffs)
    #print("---------------")
    C.<z1,z2>=PolynomialRing(CC,order='negdeglex')
    q=coeffpoly(optimal_coeffs,whichOA,mlist)
    opa=opa+[q]
    tonorm=q*f-1 
    optimaldistsq=InnerProd(tonorm,tonorm,mlist)
    OptNorms=OptNorms+[optimaldistsq]

In [17]:
#displays the OPAs prettily
for counter in range(0,len(opa)):
    show(opa[counter])

-0.666666666666667*I

-0.750000000000000*I + (-0.250000000000000)*z1

-0.823529411764706*I + (-0.235294117647059)*z1 + (-0.235294117647059)*z2

-0.834080717488789*I + (-0.269058295964126)*z1 + (-0.233183856502242)*z2 + 0.0896860986547085*I*z1^2

-0.879843060323688*I + (-0.335458558116724)*z1 + (-0.304070622854340)*z2 + 0.0784698381559588*I*z1^2 + 0.200098087297695*I*z1*z2

...

-0.968561507947532*I + (-0.452842261921299)*z1 + (-0.452842261921299)*z2 + 0.197657341393390*I*z1^2 + 0.418729067383624*I*z1*z2 + 0.197657341393389*I*z2^2 + 0.0754142860998498*z1^3 + 0.274080009850832*z1^2*z2 + 0.274080009850832*z1*z2^2 + 0.0754142860998498*z2^3 + (-0.0215159314537884*I)*z1^4 + (-0.144109590377788*I)*z1^3*z2 + (-0.236491178373036*I)*z1^2*z2^2 + (-0.144109590377787*I)*z1*z2^3 + (-0.0215159314537883*I)*z2^4 + (-0.00206182625193166)*z1^5 + (-0.0591264771984773)*z1^4*z2 + (-0.152711552897615)*z1^3*z2^2 + (-0.152711552897615)*z1^2*z2^3 + (-0.0591264771984773)*z1*z2^4 + (-0.00206182625193165)*z2^5 + (-0.00181787853412259*I)*z1^6 + 0.0160506644353679*I*z1^5*z2 + 0.0730899349032618*I*z1^4*z2^2 + 0.110362970086807*I*z1^3*z2^3 + 0.0730899349032617*I*z1^2*z2^4 + 0.0160506644353678*I*z1*z2^5 + (-0.00181787853412259*I)*z2^6 + (-0.000829125486325939)*z1^7 + 0.00133899584971046*z1^6*z2 + 0.0212607221905537*z1^5*z2^2 + 0.0493778696842263*z1^4*z2^3 + 0.0493778696842263*z1^3*z2^4 + 0.0212607221905537*z1^2*z2^5 + 0.00133899584971046*z1*z2^6 + (-0.000829125486325938)*z2^7

-0.968561639317830*I + (-0.452842507510639)*z1 + (-0.452842410442853)*z2 + 0.197657523700366*I*z1^2 + 0.418729564735147*I*z1*z2 + 0.197657281030900*I*z2^2 + 0.0754136437877714*z1^3 + 0.274081202170262*z1^2*z2 + 0.274079976447320*z1*z2^2 + 0.0754142385701018*z2^3 + (-0.0215119762383267*I)*z1^4 + (-0.144112032509752*I)*z1^3*z2 + (-0.236491295683066*I)*z1^2*z2^2 + (-0.144109508263029*I)*z1*z2^3 + (-0.0215159146400358*I)*z2^4 + (-0.00204741293465899)*z1^5 + (-0.0591308882474257)*z1^4*z2 + (-0.152711999284494)*z1^3*z2^2 + (-0.152711479533512)*z1^2*z2^3 + (-0.0591264353997047)*z1*z2^4 + (-0.00206182408181552)*z2^5 + (-0.00186177153235111*I)*z1^6 + 0.0160574782217143*I*z1^5*z2 + 0.0730908838820603*I*z1^4*z2^2 + 0.110362969669072*I*z1^3*z2^3 + 0.0730898823274679*I*z1^2*z2^4 + 0.0160506527762449*I*z1*z2^5 + (-0.00181787747098172*I)*z2^6 + (-0.000950666456272390)*z1^7 + 0.00134667803541720*z1^6*z2 + 0.0212620116224955*z1^5*z2^2 + 0.0493779764371590*z1^4*z2^3 + 0.0493778368568150*z1^3*z2^4 + 0.0212607064150306*z1^2*z2^5 + 0.00133899486042678*z1*z2^6 + (-0.000829124967065036)*z2^7 + 0.000316888818757463*I*z1^8

-0.968561709108707*I + (-0.452843405026534)*z1 + (-0.452841722299587)*z2 + 0.197660927506336*I*z1^2 + 0.418728439614353*I*z1*z2 + 0.197656720688967*I*z2^2 + 0.0754229996018009*z1^3 + 0.274080597697850*z1^2*z2 + 0.274078417916740*z1*z2^2 + 0.0754142416577511*z2^3 + (-0.0215335256822065*I)*z1^4 + (-0.144114844465785*I)*z1^3*z2 + (-0.236488290266347*I)*z1^2*z2^2 + (-0.144109222858353*I)*z1*z2^3 + (-0.0215159903843037*I)*z2^4 + (-0.00209037107765742)*z1^5 + (-0.0591446286000537)*z1^4*z2 + (-0.152707215471927)*z1^3*z2^2 + (-0.152710673374593)*z1^2*z2^3 + (-0.0591264759512994)*z1*z2^4 + (-0.00206186497303692)*z2^5 + (-0.00178982269084330*I)*z1^6 + 0.0160997245416359*I*z1^5*z2 + 0.0730845843853258*I*z1^4*z2^2 + 0.110361578285645*I*z1^3*z2^3 + 0.0730897744250468*I*z1^2*z2^4 + 0.0160507093600100*I*z1*z2^5 + (-0.00181786684955329*I)*z2^6 + (-0.000865406033345099)*z1^7 + 0.00145542915397574*z1^6*z2 + 0.0212561256404620*z1^5*z2^2 + 0.0493764348571755*z1^4*z2^3 + 0.0493775905584265*z1^3*z2^4 + 0.0212607272136493*z1^2*z2^5 + 0.00133901372979119*z1*z2^6 + (-0.000829124571482961)*z2^7 + 0.000330426246897213*I*z1^8 + (-0.000251745414693082*I)*z1^7*z2

-0.968682909560743*I + (-0.453167918854108)*z1 + (-0.452880809828122)*z2 + 0.198317123992930*I*z1^2 + 0.418944127922712*I*z1*z2 + 0.197599351427966*I*z2^2 + 0.0765528281097952*z1^3 + 0.274702688976244*z1^2*z2 + 0.274039203820109*z1*z2^2 + 0.0753501045970223*z2^3 + (-0.0231857209970031*I)*z1^4 + (-0.145506983827574*I)*z1^3*z2 + (-0.236635847150468*I)*z1^2*z2^2 + (-0.143964681745815*I)*z1*z2^3 + (-0.0215058825273408*I)*z2^4 + (-0.00399313523225431)*z1^5 + (-0.0617646915627467)*z1^4*z2 + (-0.153411526907671)*z1^3*z2^2 + (-0.152489954534074)*z1^2*z2^3 + (-0.0590856471251434)*z1*z2^4 + (-0.00206423673276400)*z2^5 + (-0.000508302801660719*I)*z1^6 + 0.0201843332827939*I*z1^5*z2 + 0.0751193676508320*I*z1^4*z2^2 + 0.110099705142550*I*z1^3*z2^3 + 0.0730182165797957*I*z1^2*z2^4 + 0.0160452561558978*I*z1*z2^5 + (-0.00181560492237487*I)*z2^6 + (-0.00136463272675485)*z1^7 + 0.00593875573091535*z1^6*z2 + 0.0261619497470511*z1^5*z2^2 + 0.0491619539476988*z1^4*z2^3 + 0.0493044721535197*z1^3*z2^4 + 0.0212470565758742*z1^2*z2^5 + 0.00134013386262229*z1*z2^6 + (-0.000828557284562005)*z2^7 + 0.000422609588072709*I*z1^8 + 0.000193607925073446*I*z1^7*z2 + (-0.0107325031468344*I)*z1^6*z2^2

-0.969866268663508*I + (-0.455495722371603)*z1 + (-0.454103083618921)*z2 + 0.201662780760857*I*z1^2 + 0.422009659499906*I*z1*z2 + 0.198181183879151*I*z2^2 + 0.0803929012926791*z1^3 + 0.280104548368242*z1^2*z2 + 0.276247606460956*z1*z2^2 + 0.0751976913075298*z2^3 + (-0.0263873418734860*I)*z1^4 + (-0.153180855427815*I)*z1^3*z2 + (-0.241780603292964*I)*z1^2*z2^2 + (-0.144422492548733*I)*z1*z2^3 + (-0.0211132007251833*I)*z2^4 + (-0.00512841753473894)*z1^5 + (-0.0702311345069475)*z1^4*z2 + (-0.162897954698802)*z1^3*z2^2 + (-0.154893374339166)*z1^2*z2^3 + (-0.0583757075476198)*z1*z2^4 + (-0.00197744959476679)*z2^5 + (-0.00164215060705671*I)*z1^6 + 0.0257556285912613*I*z1^5*z2 + 0.0893827637450506*I*z1^4*z2^2 + 0.116911896053633*I*z1^3*z2^3 + 0.0722019622453797*I*z1^2*z2^4 + 0.0158248790905299*I*z1*z2^5 + (-0.00181787725760298*I)*z2^6 + (-0.00164200261305732)*z1^7 + 0.00446494755277888*z1^6*z2 + 0.0413126927483154*z1^5*z2^2 + 0.0650402716035338*z1^4*z2^3 + 0.0486964505146113*z1^3*z2^4 + 0.0210087419068233*z1^2*z2^5 + 0.00130478071626827*z1*z2^6 + (-0.000823422538579037)*z2^7 + 0.000446524354077766*I*z1^8 + 0.000604859101648049*I*z1^7*z2 + (-0.00972156884340917*I)*z1^6*z2^2 + (-0.0338307266433815*I)*z1^5*z2^3

In [18]:
#displays the optimal norms as a list that could be easily copy pasted
#for counter in range(0,len(OptNorms)):
    #show(OptNorms[counter]) #comment this out if you just want the copy-pastable version
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
for counter in range(0,len(OptNorms)):
    print(OptNorms[counter])

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
0.333333333333333
0.250000000000000
0.176470588235294
0.165919282511211
0.120156939676312
0.112195121951220
0.111031467278331
0.0947375220010012
0.0796382984909720
0.0789842031593681
0.0788883613765463
0.0746731778160564
0.0629150115877302
0.0593424083457160
0.0593175810908916
0.0593139419432910
0.0584768656522044
0.0526961801742846
0.0471939177923232
0.0465815148190624
0.0465812269578438
0.0465811847610455
0.0464595552374694
0.0443733675258671
0.0397376744977084
0.0378613739956561
0.0377957648368699
0.0377940187299194
0.0377937627698839
0.0377835989986108
0.0372041845839279
0.0345152404453914
0.0319194371651027
0.0314412136956398
0.0314393882314452
0.0314384920524674
0.0314383606821694
0.0314382908912931
0.0313170904392565
0.0301337313364917
0.0278552027357811
0.0264069292004903
0.0256038346692775
0.0254968696756119
0.0254943524224666
0.0254942954471601
0.0254936189522646
0.0254619301

In [19]:
#gives LaTeX output for the OPAs; you should be able to just copy paste this into Overleaf if you want
print("test function")
print("\\begin{equation*}")
print("\\scalebox{2}{\\ensuremath{f=",latex(f),"}}")
print("\\end{equation*}")
deg=0
print("\\begin{align*}")
print('p_{',deg,'}&=',latex(opa[0]),'\\\\')
for deg in range(1,len(opa)):
    print('p_{',deg,'}&=',latex(opa[deg]),'\\\\')
    print('%%%%%%%%%%%%%%%%%%%')
print("\\end{align*}")
print("")

test function
\begin{equation*}
\scalebox{2}{\ensuremath{f= 1.00000000000000i + \left(-0.500000000000000\right) z_{1} + \left(-0.500000000000000\right) z_{2} }}
\end{equation*}
\begin{align*}
p_{ 0 }&= -0.666666666666667i \\
p_{ 1 }&= -0.750000000000000i + \left(-0.250000000000000\right) z_{1} \\
%%%%%%%%%%%%%%%%%%%
p_{ 2 }&= -0.823529411764706i + \left(-0.235294117647059\right) z_{1} + \left(-0.235294117647059\right) z_{2} \\
%%%%%%%%%%%%%%%%%%%
p_{ 3 }&= -0.834080717488789i + \left(-0.269058295964126\right) z_{1} + \left(-0.233183856502242\right) z_{2} + 0.0896860986547085i z_{1}^{2} \\
%%%%%%%%%%%%%%%%%%%
p_{ 4 }&= -0.879843060323688i + \left(-0.335458558116724\right) z_{1} + \left(-0.304070622854340\right) z_{2} + 0.0784698381559588i z_{1}^{2} + 0.200098087297695i z_{1} z_{2} \\
%%%%%%%%%%%%%%%%%%%
p_{ 5 }&= -0.887804878048780i + \left(-0.331707317073171\right) z_{1} + \left(-0.331707317073171\right) z_{2} + 0.0780487804878049i z_{1}^{2} + 0.195121951219512i z_{1} z_{2} + 0.0780487

In [20]:
#LaTeX output for the optimal norms
print("\\begin{align*}")
for deg in range(0,len(OptNorms)):
    print('\\nu_{',deg,'}^2&=',latex(OptNorms[deg]),'\\\\')
    print('%%%%%%%%%%%%%%%%%%%')
print("\\end{align*}")

\begin{align*}
\nu_{ 0 }^2&= 0.333333333333333 \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 1 }^2&= 0.250000000000000 \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 2 }^2&= 0.176470588235294 \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 3 }^2&= 0.165919282511211 \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 4 }^2&= 0.120156939676312 \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 5 }^2&= 0.112195121951220 \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 6 }^2&= 0.111031467278331 \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 7 }^2&= 0.0947375220010012 \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 8 }^2&= 0.0796382984909720 \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 9 }^2&= 0.0789842031593681 \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 10 }^2&= 0.0788883613765463 \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 11 }^2&= 0.0746731778160564 \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 12 }^2&= 0.0629150115877302 \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 13 }^2&= 0.0593424083457160 \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 14 }^2&= 0.0593175810908916 \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 15 }^2&= 0.0593139419432910 \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 16 }^2&= 0.0584768656522044 \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 17 }^2&= 0.0526961801742846

### Modified Code for Checking error:

In [9]:
def Convert(lst):
    return [ -i for i in lst ]

In [10]:
#Computing the OPAs and optimal norms
f = p5
howmany=15 #how many optimal approximants you need to compute
opa=[]
OptNorms=[]
for whichOA in range(0,howmany):
    JAMmat=buildMatrix(mlist,f,whichOA)
    #print(JAMmat)
    optimal_coeffs=(list(JAMmat.solve_right(RHSmateq(f,whichOA))))
    optimal_coeffs = Convert(optimal_coeffs) #change the sign of each element of optimal_coeffs ------- adding this line
    #print(optimal_coeffs)
    #print("---------------")
    C.<z1,z2>=PolynomialRing(CC,order='negdeglex')
    q=coeffpoly(optimal_coeffs,whichOA,mlist)
    opa=opa+[q]
    tonorm=q*f-1 #I think the problem is somewhere here???
    optimaldistsq=InnerProd(tonorm,tonorm,mlist)
    OptNorms=OptNorms+[optimaldistsq]